In [1]:
import os
import math
import shutil
import numpy as np
from keras.optimizers import RMSprop, SGD
from keras.losses import categorical_crossentropy
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import *
from keras.layers import *
import pandas as pd

Using TensorFlow backend.


In [2]:
%run '../lib.ipynb'
%run '../data-generator.ipynb'

In [3]:
# make only specific GPU to be utilized
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [4]:
HOLDOUT_PREDS = 'out/holdout/holdout_predictions_%d.npy'
HOLDOUT_Y = 'out/holdout/holdout_Y.npy'
FOLDS = 10

In [5]:
# read holdout predictions
h_preds = np.zeros((FOLDS, ) + np.load(HOLDOUT_PREDS % 0).shape)

for fold in range(FOLDS):
    h_preds[fold] = np.load(HOLDOUT_PREDS % fold)

h_preds = np.swapaxes(h_preds, 0, 1)
print('h_preds.shape:', h_preds.shape) # i, fold, label_1h

# read holdout Ys
holdout_Y = np.load(HOLDOUT_Y)

h_preds.shape: (4000, 10, 12)


In [6]:
# create model

i = Input(shape=(h_preds.shape[1:]))
x = Flatten()(i)
x = Dense(512, activation='sigmoid')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='sigmoid')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='sigmoid')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='sigmoid')(x)
x = Dropout(0.5)(x)
x = Dense(h_preds.shape[-1], activation='softmax')(x)

model = Model(inputs=[i], outputs=[x])
model.compile(
    optimizer=RMSprop(lr=1e-3),
    loss=categorical_crossentropy,
    metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 12)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               61952     
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
__________

In [7]:
model.fit(
    x=h_preds,
    y=holdout_Y,
    epochs=1000,
    callbacks=[
        ReduceLROnPlateau(
            monitor='loss', factor=0.2, patience=3, min_lr=1e-11, verbose=1),
        EarlyStopping(
            monitor='loss',
            min_delta=0.0000001,
            patience=20,
            verbose=1,
            mode='auto')
    ])

Epoch 1/1000
4000/4000 [==============================] - 0s - loss: 2.5937 - acc: 0.0860     
Epoch 2/1000
4000/4000 [==============================] - 0s - loss: 2.2374 - acc: 0.2303     
Epoch 3/1000
4000/4000 [==============================] - 0s - loss: 1.4469 - acc: 0.5647     
Epoch 4/1000
4000/4000 [==============================] - 0s - loss: 0.8884 - acc: 0.8015     
Epoch 5/1000
4000/4000 [==============================] - 0s - loss: 0.6147 - acc: 0.8930     
Epoch 6/1000
4000/4000 [==============================] - 0s - loss: 0.4758 - acc: 0.9220     
Epoch 7/1000
4000/4000 [==============================] - 0s - loss: 0.3958 - acc: 0.9375     
Epoch 8/1000
4000/4000 [==============================] - 0s - loss: 0.3523 - acc: 0.9435     
Epoch 9/1000
4000/4000 [==============================] - 0s - loss: 0.3389 - acc: 0.9463     
Epoch 10/1000
4000/4000 [==============================] - 0s - loss: 0.3163 - acc: 0.9485     
Epoch 11/1000
4000/4000 [========================

3552/4000 [=========================>....] - ETA: 0s - loss: 0.2704 - acc: 0.9583
Epoch 00074: reducing learning rate to 2.5600002118153498e-09.
4000/4000 [==============================] - 0s - loss: 0.2565 - acc: 0.9600     
Epoch 76/1000
4000/4000 [==============================] - 0s - loss: 0.2484 - acc: 0.9627     
Epoch 77/1000
4000/4000 [==============================] - 0s - loss: 0.2445 - acc: 0.9622     
Epoch 78/1000
3584/4000 [=========================>....] - ETA: 0s - loss: 0.2382 - acc: 0.9615
Epoch 00077: reducing learning rate to 5.1200004236307e-10.
4000/4000 [==============================] - 0s - loss: 0.2431 - acc: 0.9615     
Epoch 79/1000
4000/4000 [==============================] - 0s - loss: 0.2373 - acc: 0.9660     
Epoch 80/1000
4000/4000 [==============================] - 0s - loss: 0.2417 - acc: 0.9607     
Epoch 00079: early stopping


In [8]:
TEST_Y_FILE = 'out/test/test_predictions_%d.npy'
FNAMES_FILE = 'out/test/test_files.npy'
OUT_FILE = 'out/submission-v2-L2-B.csv'

In [10]:
# read predictions
preds_shape = np.load(TEST_Y_FILE % 0).shape
preds = np.zeros((FOLDS, ) + preds_shape)

for fold in range(FOLDS):
    preds[fold] = np.load(TEST_Y_FILE % fold)

preds = np.swapaxes(preds, 0, 1)

In [11]:
test_preds = model.predict(preds, verbose=1)

157984/158538 [============================>.] - ETA: 0s

In [12]:
fnames = np.load(FNAMES_FILE)

In [13]:
labels = [''] * len(fnames)

for i in range(len(labels)):
    labels[i]= LABELS[np.argmax(test_preds[i])]

In [14]:
df = pd.DataFrame.from_dict({'fname': fnames, 'label': labels})
df = df.set_index('fname')

In [15]:
df.to_csv(OUT_FILE, index='fname')